In [ ]:
#@title Default title text
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [8]:
### To run the script simply replace the embedding data path with the path to the embedding data in your pc and run all the cells in this script. To obtain the same results pubblished make sure that the libraries listed below are the same versions as stated here
### Running time: ~ 15'
 
import pandas as pd
import numpy as np
import scipy
import sklearn
import sklearn.linear_model
import sklearn.ensemble

print('pandas version', pd.__version__)       # version 1.1.0 was used in the paper
print('numpy version', np.__version__)        # version 1.19.1 was used in the paper
print('scipy version', scipy.__version__)     #  version 1.4.1 was used in the paper
print('sklearn version', sklearn.__version__) # 0.23.1 was used in the paper



pandas version 1.1.0
numpy version 1.18.5
scipy version 1.4.1
sklearn version 0.23.1


In [10]:
# TODO: Change to final paths when available.
CELLPROFILER_DATA_PATH = '~/cellprofiler_features_normalized_well_mean.h5'
METADATA_PATH = '~/Schiff et al. Supplementary Tables.xlsx'

In [11]:
CellProfiler_df = pd.read_hdf(CELLPROFILER_DATA_PATH)
CellProfiler_df

,,,,,cells_AreaShape_Area,cells_AreaShape_Center_X,cells_AreaShape_Center_Y,cells_AreaShape_Center_Z,cells_AreaShape_Compactness,cells_AreaShape_Eccentricity,cells_AreaShape_EulerNumber,cells_AreaShape_Extent,cells_AreaShape_FormFactor,cells_AreaShape_MajorAxisLength,...,nuclei_Texture_Variance_RNA_10_02,nuclei_Texture_Variance_RNA_10_03,nuclei_Texture_Variance_RNA_3_00,nuclei_Texture_Variance_RNA_3_01,nuclei_Texture_Variance_RNA_3_02,nuclei_Texture_Variance_RNA_3_03,nuclei_Texture_Variance_RNA_5_00,nuclei_Texture_Variance_RNA_5_01,nuclei_Texture_Variance_RNA_5_02,nuclei_Texture_Variance_RNA_5_03
batch,plateset,plate,well,cell_line_id,,,,,,,,,,,,,,,,,,,,,
1,0,01,B03,95,-0.014402,-0.006589,0.005971,0.0,-0.014558,-0.008185,-0.012663,0.017386,0.022379,-0.018125,...,-0.006587,-0.006605,-0.006648,-0.006618,-0.006640,-0.006641,-0.006637,-0.006589,-0.006622,-0.006629
2,0,01,B03,95,-0.018674,0.005994,-0.012132,0.0,-0.019783,-0.013436,-0.009278,0.025610,0.034168,-0.024928,...,0.000212,0.000185,0.000234,0.000234,0.000245,0.000218,0.000214,0.000220,0.000225,0.000203
3,0,01,B03,95,-0.022017,-0.001533,-0.010276,0.0,-0.021137,-0.020005,0.003761,0.023600,0.038103,-0.029954,...,-0.000450,-0.000458,-0.000417,-0.000428,-0.000411,-0.000436,-0.000441,-0.000445,-0.000437,-0.000457
4,0,01,B03,95,-0.005963,-0.001369,0.000310,0.0,-0.018417,-0.028806,-0.027224,0.026765,0.021630,-0.013929,...,-0.000586,-0.000545,-0.000597,-0.000603,-0.000596,-0.000593,-0.000601,-0.000604,-0.000598,-0.000589
1,0,02,B03,95,-0.014201,0.032121,-0.029059,0.0,-0.026708,-0.013273,-0.016803,0.037246,0.043757,-0.023713,...,0.005217,0.005060,0.005421,0.005394,0.005403,0.005387,0.005389,0.005330,0.005363,0.005316
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,0,05,E07,20,0.000836,0.005088,0.016994,0.0,-0.001352,0.001683,0.009896,0.001927,-0.006940,0.001420,...,0.000135,0.000164,0.000118,0.000116,0.000121,0.000120,0.000117,0.000116,0.000120,0.000120
1,0,06,E07,20,0.000238,0.013216,-0.017320,0.0,0.007510,0.009410,0.010039,-0.007737,-0.004583,0.005370,...,-0.000636,-0.000516,-0.000766,-0.000749,-0.000762,-0.000746,-0.000739,-0.000706,-0.000736,-0.000696
2,0,06,E07,20,0.006441,0.010526,0.002617,0.0,0.012494,0.009188,-0.000603,-0.014989,-0.013782,0.012495,...,-0.009205,-0.009031,-0.009274,-0.009289,-0.009278,-0.009246,-0.009264,-0.009272,-0.009270,-0.009207


In [12]:
all_metadata_df = pd.read_excel(METADATA_PATH)
all_metadata_df

,Cell line ID,Donor ID,Cross-val fold,Pair ID,Disease state,Sex,Age,European ancestry,UPDRS score,Biopsy collection year,Biopsy location,Thaw format,Thaw freeze date,Doubling time
0,01,50121,1.0,0.0,Healthy,F,54,0.99,,2012,unspecified,6w,2019-06-17,3.316818
1,02,51255,1.0,0.0,LRRK2 PD,F,56,0.92,71,2017,left upper thigh,6w,2019-06-17,2.656416
2,03,51260,2.0,1.0,Healthy,F,64,0.98,,2017,left upper leg,12w→6w,2019-08-09,2.704582
3,04,51253,2.0,1.0,Sporadic PD,F,63,0.99,23,2017,left upper arm,6w,2019-07-30,3.411927
4,05,50114,4.0,2.0,Healthy,M,67,0.94,,2012,unspecified,6w,2019-07-31,2.773009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,92,51239,NaN,NaN,Healthy,M,67,0.99,,2019,right arm,6w,2019-07-31,2.329057
92,93,51093,NaN,NaN,Healthy,F,77,0.99,,2019,left arm,6w,2019-06-19,3.080005
93,94,51148,NaN,NaN,Healthy,M,65,0.90,,2019,upper arm,6w,2019-08-13,2.210402
94,95,50492,NaN,NaN,LRRK2 PD,M,51,0.91,,2019,left upper arm,6w,2019-07-18,3.553737


In [13]:
def clean_up_relevant_metadata(all_metadata_df):
  """Make a clean version of the relevant metadata columns we need to join with the embedding dataframe."""
  # Select the columns from the metadata that we want to merge in with the embeddings.
  metadata_to_merge_df = all_metadata_df[['Cell line ID', 'Cross-val fold', 'Disease state']].copy()

  # Rename the columns.
  metadata_to_merge_df.columns = ['cell_line_id', 'group', 'disease_state']

  # Make the disease state uppercase.
  metadata_to_merge_df['disease_state'] = metadata_to_merge_df['disease_state'].str.upper()

  # Fix the naming of the disease_state.
  metadata_to_merge_df['disease_state'].replace('SPORADIC PD', 'SPORADIC', inplace=True)
  metadata_to_merge_df['disease_state'].replace('LRRK2 PD','LRRK2', inplace=True)
  metadata_to_merge_df['disease_state'].replace('HEALTHY*','HEALTHY', inplace=True)  # May not be healthy. See paper for details.

  # Strip off the whitespace on the cell line id.
  metadata_to_merge_df.loc[:, 'cell_line_id'] = metadata_to_merge_df['cell_line_id'].str.strip()

  # Remove the * from some of the cell_line_id so we can join.
  metadata_to_merge_df['cell_line_id'].replace('48*', '48', inplace=True)
  metadata_to_merge_df['cell_line_id'].replace('57*', '57', inplace=True)

  # Drop any rows that do not have a cross validation group set. This drops all the GBA lines.
  metadata_to_merge_df.dropna(axis=0, subset=['group'], inplace=True)

  # Make cross validation group an int.
  metadata_to_merge_df['group'] = metadata_to_merge_df['group'].astype(int)

  return metadata_to_merge_df

metadata_to_merge_df = clean_up_relevant_metadata(all_metadata_df)
metadata_to_merge_df

,cell_line_id,group,disease_state
0,01,1,HEALTHY
1,02,1,LRRK2
2,03,2,HEALTHY
3,04,2,SPORADIC
4,05,4,HEALTHY
...,...,...,...
85,86,3,SPORADIC
86,87,3,HEALTHY
87,88,3,SPORADIC
88,89,2,HEALTHY


In [14]:
def merge_metadata_with_CellProfiler_df(CellProfiler_df, metadata_to_merge_df):
  """Join the metadata with the embedding dataframe."""
  # Reset the index of the embedding dataframe before merge.
  CellProfiler_to_merge_df = CellProfiler_df.reset_index()

  # Merge the embedding data with the metadata.
  merged_df = pd.merge(metadata_to_merge_df, CellProfiler_to_merge_df, on='cell_line_id')

  return merged_df

train_test_df = merge_metadata_with_CellProfiler_df(CellProfiler_df, metadata_to_merge_df)
train_test_df

,cell_line_id,group,disease_state,batch,plateset,plate,well,cells_AreaShape_Area,cells_AreaShape_Center_X,cells_AreaShape_Center_Y,...,nuclei_Texture_Variance_RNA_10_02,nuclei_Texture_Variance_RNA_10_03,nuclei_Texture_Variance_RNA_3_00,nuclei_Texture_Variance_RNA_3_01,nuclei_Texture_Variance_RNA_3_02,nuclei_Texture_Variance_RNA_3_03,nuclei_Texture_Variance_RNA_5_00,nuclei_Texture_Variance_RNA_5_01,nuclei_Texture_Variance_RNA_5_02,nuclei_Texture_Variance_RNA_5_03
0,01,1,HEALTHY,1,1,07,D03,0.000334,0.005393,0.007596,...,-0.010826,-0.010723,-0.010949,-0.010941,-0.010947,-0.010919,-0.010919,-0.010898,-0.010916,-0.010867
1,01,1,HEALTHY,2,1,07,D03,-0.006525,-0.005103,0.001584,...,-0.004697,-0.004559,-0.004881,-0.004857,-0.004875,-0.004844,-0.004837,-0.004787,-0.004832,-0.004772
2,01,1,HEALTHY,3,1,07,D03,-0.005115,-0.001606,0.007452,...,-0.004095,-0.004073,-0.004177,-0.004166,-0.004157,-0.004155,-0.004171,-0.004144,-0.004142,-0.004134
3,01,1,HEALTHY,4,1,07,D03,-0.004642,0.000651,-0.003254,...,0.000095,0.000140,0.000073,0.000072,0.000070,0.000063,0.000070,0.000080,0.000064,0.000065
4,01,1,HEALTHY,1,1,08,D03,-0.001372,0.004322,0.004148,...,-0.011720,-0.011607,-0.011876,-0.011861,-0.011865,-0.011851,-0.011854,-0.011811,-0.011839,-0.011797
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3528,90,2,SPORADIC,3,0,05,A09,0.025981,-0.009565,-0.002940,...,0.004716,0.004681,0.004893,0.004847,0.004884,0.004827,0.004813,0.004772,0.004805,0.004739
3529,90,2,SPORADIC,4,0,05,A09,0.008228,-0.028650,0.013530,...,0.000491,0.000417,0.000517,0.000522,0.000524,0.000523,0.000517,0.000513,0.000528,0.000513
3530,90,2,SPORADIC,1,0,06,A09,0.021462,-0.015928,0.001214,...,0.012258,0.012290,0.012266,0.012259,0.012294,0.012219,0.012207,0.012231,0.012245,0.012173
3531,90,2,SPORADIC,3,0,06,A09,0.008020,-0.008984,-0.019546,...,0.024134,0.024219,0.024229,0.024177,0.024224,0.024197,0.024170,0.024130,0.024160,0.024154


In [15]:
def run_prediction_model(data, model_name):
  """Train the model and return the prediction results."""
  Results = pd.DataFrame(
      columns=['group0', 'group1', 'group2', 'group3', 'group4', 'mean', 'std'])

  for fold in np.unique(data.group):
    auxTrain = data.loc[data.group != fold]
    aux_test = data.loc[data.group == fold]
    auxTrain = auxTrain.sort_values(axis=0, by=['batch', 'plate', 'well'])
    X_train = auxTrain.iloc[:,7:]
    Targ = auxTrain.disease_state
    Targ = Targ.replace('HEALTHY', 0)
    Targ = Targ.replace('SPORADIC', 1)
    y_train = Targ.replace('LRRK2', 1)

    aux_test = aux_test.sort_values(axis=0, by=['batch', 'plate', 'well'])
    X_test = aux_test.iloc[:,7:]
    Targ = aux_test.disease_state
    Targ = Targ.replace('HEALTHY', 0)
    Targ = Targ.replace('SPORADIC', 1)
    y_test = Targ.replace('LRRK2', 1)

    if model_name == 'LogisticCV':
      RF = sklearn.linear_model.LogisticRegressionCV(
          solver='lbfgs', max_iter=1000000)
    elif model_name == 'Logistic':
      RF = sklearn.linear_model.LogisticRegression(solver='lbfgs')
    elif model_name == 'RidgeCV':
      RF = sklearn.linear_model.RidgeCV()
    else:
      raise ValueError('Unknown model_name: %s' % model_name)

    RF.fit(X_train, y_train)
    rf_predictions = RF.predict(X_test)
    if model_name == 'RidgeCV':
      preds = RF.predict(X_test)
    else:
      preds = RF.predict_proba(X_test)[:, 1]
    pred_df = pd.DataFrame(data=preds, index=X_test.index, columns=['pred'])
    pred_df['cell_line_id'] = aux_test.cell_line_id
    pred_arr = pred_df.groupby('cell_line_id').mean()
    pred_df['disease_state'] = aux_test.disease_state
    pred_arr = pred_df.groupby(['cell_line_id', 'disease_state']).mean()
    pred_arr = pred_arr.reset_index()
    pred_arr['true'] = 0
    for line in pred_arr.cell_line_id:
      if aux_test.loc[aux_test.cell_line_id == line,
                      'disease_state'].values[0] != 'HEALTHY':
        pred_arr.loc[pred_arr.cell_line_id == line, 'true'] = 1
    Results.loc[
        'spoAUC', 'group' + str(int(fold))] = sklearn.metrics.roc_auc_score(
            pred_arr.drop(
                pred_arr.loc[pred_arr.disease_state == 'LRRK2'].index).true,
            pred_arr.drop(
                pred_arr.loc[pred_arr.disease_state == 'LRRK2'].index).pred)
    Results.loc[
        'LrrkAUC', 'group' + str(int(fold))] = sklearn.metrics.roc_auc_score(
            pred_arr.drop(
                pred_arr.loc[pred_arr.disease_state == 'SPORADIC'].index).true,
            pred_arr.drop(
                pred_arr.loc[pred_arr.disease_state == 'SPORADIC'].index).pred)
    Results.loc['totalAUC',
                'group' + str(int(fold))] = sklearn.metrics.roc_auc_score(
                    pred_arr.true, pred_arr.pred)

  Results.loc['spoAUC', 'mean'] = np.mean(Results.loc['spoAUC'].values[0:-2])
  Results.loc['spoAUC', 'std'] = np.std(Results.loc['spoAUC'].values[0:-2])
  Results.loc['LrrkAUC', 'mean'] = np.mean(Results.loc['LrrkAUC'].values[0:-2])
  Results.loc['LrrkAUC', 'std'] = np.std(Results.loc['LrrkAUC'].values[0:-2])

  Results.loc['totalAUC',
              'mean'] = np.mean(Results.loc['totalAUC'].values[0:-2])
  Results.loc['totalAUC', 'std'] = np.std(Results.loc['totalAUC'].values[0:-2])
  return Results


In [16]:
logistic_cv_result = run_prediction_model(train_test_df,'LogisticCV')
logistic_cv_result

,group0,group1,group2,group3,group4,mean,std
spoAUC,0.714286,0.571429,0.714286,0.547619,0.553571,0.620238,0.0771884
LrrkAUC,0.625,0.642857,0.714286,0.714286,1,0.739286,0.135338
totalAUC,0.703125,0.591837,0.714286,0.571429,0.609375,0.63801,0.0590643
